In [0]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#2. Get the file
downloaded = drive.CreateFile({'id':'145IOrHLQMkGy8moUXfLSEK4fk0UtkhED'}) # replace the id with id of file you want to access
downloaded.GetContentFile('ratings.dat')

In [0]:
#@title Default title text
import pandas as pd
import numpy as np
import os
import time
#os.chdir('C://Users//Deepanshi//Documents//competitions//png//recommendation//ml_1m//ml-1m')

In [188]:
train=pd.read_csv("ratings.dat",sep="::",header=None)
train.columns = ['user','movie','rating','timestamp']
count_movie = train.groupby('movie').count()
count_movie = count_movie.drop('rating',axis = 1)
count_movie = count_movie.drop('timestamp',axis = 1).reset_index()
movie_rated_more_than_10 = count_movie[count_movie["user"]>10]
data_temp = pd.merge(movie_rated_more_than_10, train, how='left', on='movie')
data_temp = data_temp.drop('user_x',1)
data_temp.columns = ['movie','user','rating','timestamp']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [189]:
train.shape

(1000209, 4)

In [0]:
ratings = data_temp.pivot(index = "user", columns = "movie",values = "rating")
ratings = ratings.fillna(0)
ratings = ratings.as_matrix()

In [178]:
ratings.shape

(6040, 3233)

In [0]:
from math import floor
import random

In [191]:
train_sample = random.sample(range(0,ratings.shape[0]),floor(ratings.shape[0]*0.8))
total_sample = list(range(6040))
test_sample = list(set(total_sample) - set(train_sample))
train_ratings = ratings[train_sample,:]
train_ratings.shape
test_ratings = ratings[test_sample,:]
test_ratings.shape

(1208, 3233)

In [181]:
train_ratings.shape

(4832, 3233)

In [0]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

In [0]:
def create_lstm_vae(input_dim, 
    batch_size, 
    intermediate_dim, 
    latent_dim,
    epsilon_std=1.):

    # Q(z|X) -- encoder
    inputs = Input(shape=(input_dim,))
    h_q = Dense(intermediate_dim, activation='relu')(inputs)
    

    # VAE Z layer
    mu = Dense(latent_dim, activation='linear')(h_q)
    log_sigma = Dense(latent_dim, activation='linear')(h_q)
    
    def sampling(args):
        mu, log_sigma = args
        eps = K.random_normal(shape=(batch_size, latent_dim),
                                  mean=0., stddev=epsilon_std)
        return mu + K.exp(log_sigma/2) * eps

    # note that "output_shape" isn't necessary with the TensorFlow backend
    
    z = Lambda(sampling, output_shape = (latent_dim,))([mu, log_sigma]) # what does this give as output? 
   
    # P(X|z) -- decoder
    decoder_hidden = Dense(intermediate_dim, activation='relu')
    decoder_out = Dense(input_dim, activation='sigmoid')
    h_p = decoder_hidden(z)
    outputs = decoder_out(h_p)
                   
    # Overall VAE model, for reconstruction and training
    vae = Model(inputs, outputs)

    # Encoder model, to encode input into latent variable
    # We use the mean as the output as it is the center point, the representative of the gaussian
    encoder = Model(inputs, mu)

    # Generator model, generate new data given latent variable z
    d_in = Input(shape=(latent_dim,))
    d_h = decoder_hidden(d_in)
    d_out = decoder_out(d_h)
    decoder = Model(d_in, d_out)

    def vae_loss(y_true, y_pred):
        """ Calculate loss = reconstruction loss + KL loss for each data in minibatch """
        # E[log P(X|z)]
        recon = K.sum(K.binary_crossentropy(y_pred, y_true), axis=1)
        # D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
        kl = 0.5 * K.sum(K.exp(log_sigma) + K.square(mu) - 1. - log_sigma, axis=1)

        return recon + kl

    vae.compile(optimizer='adam', loss=vae_loss)
    
    return vae, encoder, decoder


In [0]:
input_dim = train_ratings.shape[-1] # 13
batch_size = 1



In [0]:
vae1, enc1, gen1 = create_lstm_vae(input_dim, 
        batch_size=batch_size, 
        intermediate_dim=32,
        latent_dim=10,
        epsilon_std=1.)


In [195]:
start_time = time.time()
output_temp = vae1.fit(train_ratings, train_ratings, epochs=20)
print(time.time()-start_time)

Epoch 1/20
4832/4832 [==============================] - 3s 561us/step - loss: 6177295051.7556
Epoch 2/20
4832/4832 [==============================] - 2s 437us/step - loss: 60235171518.7983
Epoch 3/20
4832/4832 [==============================] - 2s 434us/step - loss: 2831.0180
Epoch 4/20
4832/4832 [==============================] - 2s 443us/step - loss: 2743.1760
Epoch 5/20
4832/4832 [==============================] - 2s 445us/step - loss: 2723.2269
Epoch 6/20
4832/4832 [==============================] - 2s 449us/step - loss: 2700.3632
Epoch 7/20
4832/4832 [==============================] - 2s 449us/step - loss: 2683.7836
Epoch 8/20
 416/4832 [=>............................] - ETA: 1s - loss: 2555.0255

4832/4832 [==============================] - 2s 441us/step - loss: 2684.7651
Epoch 9/20
4832/4832 [==============================] - 2s 443us/step - loss: 2667.5789
Epoch 10/20
4832/4832 [==============================] - 2s 435us/step - loss: 2662.4590
Epoch 11/20
4832/4832 [==============================] - 2s 439us/step - loss: 2655.4006
Epoch 12/20
4832/4832 [==============================] - 2s 441us/step - loss: 2650.1916
Epoch 13/20
4832/4832 [==============================] - 2s 440us/step - loss: 2645.1939
Epoch 14/20
4832/4832 [==============================] - 2s 432us/step - loss: 2638.1253
Epoch 15/20
1312/4832 [=======>......................] - ETA: 1s - loss: 2667.1477

4832/4832 [==============================] - 2s 434us/step - loss: 2632.6927
Epoch 16/20
4832/4832 [==============================] - 2s 437us/step - loss: 2629.4312
Epoch 17/20
4832/4832 [==============================] - 2s 436us/step - loss: 2624.2764
Epoch 18/20
4832/4832 [==============================] - 2s 441us/step - loss: 2616.9063
Epoch 19/20
4832/4832 [==============================] - 2s 440us/step - loss: 2611.1288
Epoch 20/20
4832/4832 [==============================] - 2s 434us/step - loss: 2605.2128
43.54880332946777


In [214]:
total_score = 0
count = 0
count_2 = 0
n = 50
for row in train_ratings:
  temp_row_1 = row.copy()
  temp_row_2 = row.copy()
  count = count+1
  non_zero_indices = temp_row_2.nonzero()
  n_non_zero_indices = len(non_zero_indices[0])
  temp = (temp_row_2>3).sum()
  if(n_non_zero_indices>n):
    count_2 += 1
    row_new = np.array(temp_row_2)[np.newaxis]
    preds = vae1.predict(row_new, batch_size=batch_size)
    preds = preds.reshape(3233,)
    top_40_orig = np.argpartition(temp_row_1, -temp)[-temp:]
    top_40_pred = np.argpartition(preds, -n)[-n:]
    score = len(set(top_40_orig).intersection(set(top_40_pred)))
    score = score/n
    total_score = total_score+score
print(total_score/count_2)

0.2865005931198103
